In [6]:
import pandas as pd
import os
import glob
import dask

In [7]:
filenames = []
for file in os.listdir("./flights_month/"):
    filenames.append("./flights_month/"+file)


In [8]:
filenames

['./flights_month/flight_12.csv',
 './flights_month/flight_8.csv',
 './flights_month/flight_2.csv',
 './flights_month/flight_7.csv',
 './flights_month/flight_3.csv',
 './flights_month/flight_11.csv',
 './flights_month/flight_10.csv',
 './flights_month/flight_5.csv',
 './flights_month/flight_9.csv',
 './flights_month/flight_4.csv',
 './flights_month/flight_6.csv',
 './flights_month/flight_1.csv']

In [9]:
%%time

sums = []
counts = []
for fn in filenames:
    # Read in file
    df = pd.read_csv(fn)

    # Groupby origin airport
    by_origin = df.groupby("origin")

    # Sum of all departure delays by origin
    total = by_origin.dep_delay.sum()

    # Number of flights by origin
    count = by_origin.dep_delay.count()

    # Save the intermediates
    sums.append(total)
    counts.append(count)

# Combine intermediates to get total mean-delay-per-origin
total_delays = sum(sums)
n_flights = sum(counts)
mean = total_delays / n_flights

CPU times: user 309 ms, sys: 51.1 ms, total: 360 ms
Wall time: 360 ms


In [10]:
%%time

@dask.delayed
def read_file(filename):
    # Read in file
    df = pd.read_csv(filename)

    # Groupby origin airport
    by_origin = df.groupby("origin")

    # Sum of all departure delays by origin
    total = by_origin.dep_delay.sum()

    # Number of flights by origin
    count = by_origin.dep_delay.count()

    return (total, count)


#sums = dask.delayed([])
#counts = dask.delayed([])
#total = dask.delayed(int)
#count = dask.delayed(int)

sums = []
counts = []
for fn in filenames:
    # Delayed read in file
    total, count = read_file(fn).compute()

    # Save the intermediates
    sums.append(total)
    counts.append(count)

# Combine intermediates to get total mean-delay-per-origin
total_delays = sum(sums)
n_flights = sum(counts)
mean, *_ = dask.compute(total_delays / n_flights)

CPU times: user 377 ms, sys: 93.5 ms, total: 471 ms
Wall time: 469 ms
